# IBM Data Science Capstone

## Districts of Moscow. Data Analysis

### By Bogdan S.
----------------------

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## *Introduction/Business Problem* <a name="introduction"></a>

![alt text](https://i.weheart.moscow/w/2017/05/07173451/moscow-skyline.jpg)

### As each big city Moscoow has districts that differs from each other by their accommodation, prestigiousness and house values. The modern division of the city was set during the Soviet period of our history. The idea of my research is to look, which types of venues are popular within different parts of Moscow and see which reccomendations can be given to stakeholders that wants to set up a new place in Moscow: where and what should be done to attract local people.
--------

## *Data* <a name="data"></a>

### 1. Foursquare location data.   
As I observed earlier Foursquare has a lot of Data about venues in Moscow, that will help us to examine most popular venue types in each district.
![alt text](http://www.orangesmile.com/common/img_city_maps/moscow-map-0.jpg)
### 2. Moscow administrative data.
As many big cities all around the world Moscow has a lot of administrative data in open admission, which will help us to distinguish districts relying on its administrative quality.
![alt text](https://www.researchgate.net/profile/Ilya_Kashnitsky/publication/303847650/figure/fig1/AS:372645280403457@1465856966513/Descriptive-maps-of-Moscow-districts-A-reference-map-of-administrative-division-B.png)
### 3. Structured and unstructured data.
Another our task is to collect (and structure) data which will us represent prestigiousness of districts: average price of an apartment, average price of a square metre of an apartment and so on.
![alt text](https://tech.eu/wp-content/uploads/2014/08/Moscow-Vacation-Rentals-Short-Term-Rentals-Airbnb1-1024x400.png)

## *Analysis* <a name="analysis"></a>

In [1]:
#pandas library to maintain dataframes
import pandas as pd
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# import geocoder
!pip install geocoder
import geocoder
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

     |████████████████████████████████| 102kB 29.8MB/s ta 0:00:01
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0  

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
#looking for geographical coordinates of Moscow
address = 'Moscow, Russia'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Moscow are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Moscow are 55.7504461, 37.6174943.


### Administrative districts of Moscow

In [4]:
# create map of Moscow using latitude and longitude values
map_moscow = folium.Map(location=[latitude, longitude], zoom_start=10)
#getting coordinates of Moscow administrative districts

ad='http://gis-lab.info/data/mos-adm/ao.geojson'
folium.GeoJson(
    ad,
    name='geojson',
    style_function = lambda x: {'fillColor': '#00000000'}
).add_to(map_moscow)
map_moscow

In [5]:
#FourSquare query
LIMIT = 15000 # limit of number of venues returned by Foursquare API

radius = 20000 # define radius
categoryId = "4bf58dd8d48988d1d2941735"
 # create URL
url_suchi = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT,
    categoryId)
url_suchi # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=2EJYSN2D5C0X4JA4PZVPKXKSEGYZZMEP11FZ32S2FWMDFQOZ&client_secret=J2X1NSROUWBNUR32T4ZVVIYG31FOXGFDIWXXGIH2TOCJWTXT&v=20180605&ll=55.7504461,37.6174943&radius=20000&limit=15000&categoryId=4bf58dd8d48988d1d2941735'

In [9]:
results_suchi = requests.get(url_suchi).json()


### Let's regard 5 popular types of venues in Moscow: Sushi bar, Coffee shop, Pizzeria, French cuisine restaurant and Fastfood

In [12]:
#let's start with sushi bars
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
sushi = results_suchi['response']['groups'][0]['items']
df_sushi = json_normalize(sushi)
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
df_sushi =df_sushi.loc[:, filtered_columns]
# filter the category for each row

df_sushi['venue.categories'] = df_sushi.apply(get_category_type, axis=1)

# clean columns
df_sushi.columns = [col.split(".")[-1] for col in df_sushi.columns]
df_sushi.head()

,name,categories,lat,lng
0,Corner Café & Kitchen,Japanese Restaurant,55.751496,37.586757
1,Cosmoryba,Sushi Restaurant,55.727587,37.567575
2,Novikov,Asian Restaurant,55.757512,37.612967
3,Ки-До,Sushi Restaurant,55.747378,37.597697
4,Кафе и суши-бар фитнес-клуба «Онѣгинъ»,Sushi Restaurant,55.736510,37.616371


In [13]:
for i in range(0,len(df_sushi)):
    folium.CircleMarker(
        [df_sushi.iloc[i]['lat'], df_sushi.iloc[i]['lng']],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_moscow)


In [14]:
map_moscow

In [15]:
#let's continue with pizzerias
#FourSquare query
LIMIT = 15000 # limit of number of venues returned by Foursquare API

radius = 20000 # define radius
categoryId = "4bf58dd8d48988d1ca941735"
 # create URL
url_pizza = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT,
    categoryId)
url_pizza # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=2EJYSN2D5C0X4JA4PZVPKXKSEGYZZMEP11FZ32S2FWMDFQOZ&client_secret=J2X1NSROUWBNUR32T4ZVVIYG31FOXGFDIWXXGIH2TOCJWTXT&v=20180605&ll=55.7504461,37.6174943&radius=20000&limit=15000&categoryId=4bf58dd8d48988d1ca941735'

In [16]:
results_pizza = requests.get(url_pizza).json()

In [17]:

pizza = results_pizza['response']['groups'][0]['items']
df_pizza = json_normalize(pizza)

df_pizza =df_pizza.loc[:, filtered_columns]
# filter the category for each row

df_pizza['venue.categories'] = df_pizza.apply(get_category_type, axis=1)

# clean columns
df_pizza.columns = [col.split(".")[-1] for col in df_pizza.columns]
df_pizza.head()

,name,categories,lat,lng
0,Pinzeria by Bontempi,Pizza Place,55.746038,37.602141
1,Camorra Pizza e Birra,Pizza Place,55.755135,37.636913
2,Pinsa Maestrello,Pizza Place,55.758779,37.645721
3,Zotman Pizza Pie,Pizza Place,55.757257,37.599035
4,BRO&N Pizzeria,Pizza Place,55.763546,37.593872


In [18]:
# create map of Moscow using latitude and longitude values
map_moscow_pizza = folium.Map(location=[latitude, longitude], zoom_start=10)
#getting coordinates of Moscow administrative districts

ad='http://gis-lab.info/data/mos-adm/ao.geojson'
folium.GeoJson(
    ad,
    name='geojson',
    style_function = lambda x: {'fillColor': '#00000000'}
).add_to(map_moscow_pizza)


In [19]:
for i in range(0,len(df_pizza)):
    folium.CircleMarker(
        [df_pizza.iloc[i]['lat'], df_pizza.iloc[i]['lng']],
        radius=5,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_moscow_pizza)
map_moscow_pizza

In [20]:
#next type of venue are coffee shops

#FourSquare query
LIMIT = 15000 # limit of number of venues returned by Foursquare API

radius = 20000 # define radius
categoryId = "4bf58dd8d48988d1e0931735"
 # create URL
url_coffee = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT,
    categoryId)
url_coffee # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=2EJYSN2D5C0X4JA4PZVPKXKSEGYZZMEP11FZ32S2FWMDFQOZ&client_secret=J2X1NSROUWBNUR32T4ZVVIYG31FOXGFDIWXXGIH2TOCJWTXT&v=20180605&ll=55.7504461,37.6174943&radius=20000&limit=15000&categoryId=4bf58dd8d48988d1e0931735'

In [21]:
results_coffee = requests.get(url_coffee).json()

In [22]:
coffee = results_coffee['response']['groups'][0]['items']
df_coffee = json_normalize(coffee)

df_coffee =df_coffee.loc[:, filtered_columns]
# filter the category for each row

df_coffee['venue.categories'] = df_coffee.apply(get_category_type, axis=1)

# clean columns
df_coffee.columns = [col.split(".")[-1] for col in df_coffee.columns]
df_coffee.head()

,name,categories,lat,lng
0,One Teaspoon,Coffee Shop,55.750781,37.643458
1,Чайных дел мастерская,Tea Room,55.761063,37.635439
2,Surf Coffee,Coffee Shop,55.767762,37.599931
3,Мойчай,Tea Room,55.763169,37.635764
4,Surf Coffee,Coffee Shop,55.757233,37.622688


In [23]:
# create map of Moscow using latitude and longitude values
map_moscow_coffee = folium.Map(location=[latitude, longitude], zoom_start=10)
#getting coordinates of Moscow administrative districts

ad='http://gis-lab.info/data/mos-adm/ao.geojson'
folium.GeoJson(
    ad,
    name='geojson',
    style_function = lambda x: {'fillColor': '#00000000'}
).add_to(map_moscow_coffee)


In [24]:
for i in range(0,len(df_coffee)):
    folium.CircleMarker(
        [df_coffee.iloc[i]['lat'], df_coffee.iloc[i]['lng']],
        radius=5,
        color='yellow',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7,
        parse_html=False).add_to(map_moscow_coffee)
map_moscow_coffee

In [25]:
#french restaurants

#FourSquare query
LIMIT = 15000 # limit of number of venues returned by Foursquare API

radius = 20000 # define radius
categoryId = "4bf58dd8d48988d10c941735"
 # create URL
url_french = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT,
    categoryId)
url_french # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=2EJYSN2D5C0X4JA4PZVPKXKSEGYZZMEP11FZ32S2FWMDFQOZ&client_secret=J2X1NSROUWBNUR32T4ZVVIYG31FOXGFDIWXXGIH2TOCJWTXT&v=20180605&ll=55.7504461,37.6174943&radius=20000&limit=15000&categoryId=4bf58dd8d48988d10c941735'

In [26]:
results_french = requests.get(url_french).json()

In [27]:
french = results_french['response']['groups'][0]['items']
df_french = json_normalize(french)

df_french =df_french.loc[:, filtered_columns]
# filter the category for each row

df_french['venue.categories'] = df_french.apply(get_category_type, axis=1)

# clean columns
df_french.columns = [col.split(".")[-1] for col in df_french.columns]
df_french.head()

,name,categories,lat,lng
0,Cristal Room Baccarat,French Restaurant,55.758704,37.624360
1,Мишель,French Restaurant,55.761497,37.571372
2,Pâté & Co,French Restaurant,55.747675,37.625656
3,Le Restaurant,French Restaurant,55.762420,37.554917
4,Brasserie Мост,French Restaurant,55.761441,37.617879


In [28]:
# create map of Moscow using latitude and longitude values
map_moscow_french = folium.Map(location=[latitude, longitude], zoom_start=10)
#getting coordinates of Moscow administrative districts

ad='http://gis-lab.info/data/mos-adm/ao.geojson'
folium.GeoJson(
    ad,
    name='geojson',
    style_function = lambda x: {'fillColor': '#00000000'}
).add_to(map_moscow_french)


In [29]:
for i in range(0,len(df_french)):
    folium.CircleMarker(
        [df_french.iloc[i]['lat'], df_french.iloc[i]['lng']],
        radius=5,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_moscow_french)
map_moscow_french

In [30]:
#fastfood
#FourSquare query
LIMIT = 15000 # limit of number of venues returned by Foursquare API

radius = 20000 # define radius
categoryId = "4bf58dd8d48988d16e941735"
 # create URL
url_ff = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT,
    categoryId)
url_ff # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=2EJYSN2D5C0X4JA4PZVPKXKSEGYZZMEP11FZ32S2FWMDFQOZ&client_secret=J2X1NSROUWBNUR32T4ZVVIYG31FOXGFDIWXXGIH2TOCJWTXT&v=20180605&ll=55.7504461,37.6174943&radius=20000&limit=15000&categoryId=4bf58dd8d48988d16e941735'

In [31]:
results_ff = requests.get(url_ff).json()

In [32]:
ff = results_ff['response']['groups'][0]['items']
df_ff = json_normalize(ff)

df_ff =df_ff.loc[:, filtered_columns]
# filter the category for each row

df_ff['venue.categories'] = df_ff.apply(get_category_type, axis=1)

# clean columns
df_ff.columns = [col.split(".")[-1] for col in df_ff.columns]
df_ff.head()

,name,categories,lat,lng
0,макдональдс,Fast Food Restaurant,55.754728,37.752472
1,Mcdonalds's,Fast Food Restaurant,55.801974,37.532372
2,McDonald's,Fast Food Restaurant,55.718514,37.607785
3,Burger King,Fast Food Restaurant,55.775228,37.657390
4,McDonald's,Fast Food Restaurant,55.783778,37.718356


In [33]:
# create map of Moscow using latitude and longitude values
map_moscow_ff = folium.Map(location=[latitude, longitude], zoom_start=10)
#getting coordinates of Moscow administrative districts

ad='http://gis-lab.info/data/mos-adm/ao.geojson'
folium.GeoJson(
    ad,
    name='geojson',
    style_function = lambda x: {'fillColor': '#00000000'}
).add_to(map_moscow_ff)


In [34]:
for i in range(0,len(df_ff)):
    folium.CircleMarker(
        [df_ff.iloc[i]['lat'], df_ff.iloc[i]['lng']],
        radius=5,
        color='black',
        fill=True,
        fill_color='black',
        fill_opacity=0.7,
        parse_html=False).add_to(map_moscow_ff)
map_moscow_ff

## *Results and Discussion* <a name="results"></a>

###  These maps show us, how different types of venues are distributed all around Moscow. Knowing that central, western and southern Moscow districts are more prestigious then northern and eastern Moscow, we may now suggest which type of venue is better to open in each district. We have regarded the most common type of venues because Moscow has no division based on the nationality of its inhabitants (such as China-town for example). That's why we did not pay attention to specific cuisines that are popular only inside one culture.

## *Conclusion* <a name="conclusion"></a>

#### 1. Central administrative district is a good place to open any kind of venue due to a big number of tourists visiting it and offices located nearby.
#### 2. Fastfood restaurants may be opened in any district od Moscow due to the high density ogf population.
#### 3. French restaurants should be opened mostly in the central disctrict due to its usually heavy prices.
#### 4. Coffee shops may be opened in central, western and southern Moscow due to a big number of offices located there.
#### 5. Pizzerias may opened in any distrcit but opening in the Southern administrative district is of high risk.
#### 6. Sushi bars may be opened in any district but opening in the Eastern administrative district is of high risk.

# Welcome to Moscow!


<img style="float: left;" src="http://welcome.moscow/upload/iblock/94d/94df8ddc3edbb9948c27ed825f935bf9.jpg">